In [1]:
import pandas as pd
import numpy as np
from itertools import gr

In [3]:
run_output_file_path = "ParticleDAScaling160Longer1BLASThread.o395270"

In [28]:
table_rule_line_numbers = []
with open(run_output_file_path, "r") as output_file:
    for line_number, line in enumerate(output_file):
        if line.strip(" ").startswith("─" * 50):
            table_rule_line_numbers.append(line_number)
print(f"Line numbers of table rules: {table_rule_line_numbers}")
number_of_ranks = len(table_rule_line_numbers) // 3

Line numbers of table rules: [56, 62, 211, 212, 218, 361, 362, 368, 511, 512, 518, 661]


In [60]:
timing_dataframes = {}
for rank in range(number_of_ranks):
    start_rule_line, header_rule_line, end_rule_line = table_rule_line_numbers[rank * 3:(rank + 1) * 3]
    timing_dataframes[f"Rank {rank}"] = pd.read_fwf(
        run_output_file_path,
        skiprows=list(range(0, header_rule_line - 1)) + [header_rule_line] + list(range(end_rule_line , line_number + 1)),
        header=0
    )

In [102]:
statistics = ["mean", "std", "min", "max"]
summary_stats = {key: [] for key in ["rank", "operation"] + statistics}
for rank in range(number_of_ranks):
    dataframe = timing_dataframes[f"Rank {rank}"]
    for operation in ["Up. determin.", "Up. stochastic", "Weight"]:
        timings = pd.to_timedelta(
            dataframe[dataframe.Section.str.strip().str.startswith(operation)].time
        )
        summary_stats["rank"].append(rank)
        summary_stats["operation"].append(operation)
        for stat in statistics:
            summary_stats[stat].append(getattr(timings, stat)().total_seconds())
summary_stats = pd.DataFrame(summary_stats)

In [104]:
print(summary_stats)

    rank       operation      mean       std      min     max
0      0   Up. determin.  1.202200  0.213851  0.93000  2.0200
1      0  Up. stochastic  1.046350  0.144915  0.84700  1.4900
2      0          Weight  0.006631  0.008476  0.00203  0.0416
3      1   Up. determin.  1.318625  0.776672  0.90900  4.0500
4      1  Up. stochastic  1.044950  0.181585  0.75400  1.7900
5      1          Weight  0.004057  0.005337  0.00203  0.0289
6      2   Up. determin.  1.320475  0.276602  0.92600  1.7900
7      2  Up. stochastic  1.160150  0.339650  0.74000  2.4000
8      2          Weight  0.005271  0.012202  0.00203  0.0751
9      3   Up. determin.  1.562925  0.787751  0.97700  3.8300
10     3  Up. stochastic  1.090925  0.319656  0.72500  2.8200
11     3          Weight  0.007265  0.008803  0.00211  0.0425


In [117]:
timing_dataframes["Rank 0"][timing_dataframes["Rank 0"].Section.str.strip().str.startswith("Up. det")].sort_values("Section")

,Section,ncalls,time,%tot,avg,alloc,%tot.1,avg.1
19,Up. determin. 1,1.00k,2.02s,6.3%,2.02ms,511MiB,23.5%,523KiB
21,Up. determin. 10,1.00k,1.53s,4.8%,1.53ms,399MiB,18.3%,408KiB
46,Up. determin. 11,1.00k,1.41s,4.4%,1.41ms,384MiB,17.6%,393KiB
16,Up. determin. 12,1.00k,1.40s,4.4%,1.40ms,371MiB,17.0%,380KiB
97,Up. determin. 13,1.00k,1.20s,3.8%,1.20ms,312MiB,14.4%,320KiB
18,Up. determin. 14,1.00k,1.19s,3.8%,1.19ms,323MiB,14.9%,331KiB
101,Up. determin. 15,1.00k,1.18s,3.7%,1.18ms,327MiB,15.1%,335KiB
106,Up. determin. 16,1.00k,1.17s,3.7%,1.17ms,307MiB,14.1%,315KiB
77,Up. determin. 17,1.00k,1.05s,3.3%,1.05ms,294MiB,13.5%,301KiB
47,Up. determin. 18,1.00k,1.07s,3.4%,1.07ms,278MiB,12.8%,285KiB


In [123]:
3200 / np.array([80, 160, 320, 640, 1600])

array([40., 20., 10.,  5.,  2.])

In [109]:
np.array([80, 160, 320, 640, 1600]) / 40

array([ 2.,  4.,  8., 16., 40.])